## 逻辑回归评分卡模型

In [5]:
%pwd
# %cd Practice/

'/Users/danielzhang/Documents/Python-Projects/DataAnalysis/Practice'

In [6]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression

## 数据预处理

In [7]:
data = pd.read_csv('rankingcard.csv')

In [8]:
data.head()

,index,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
1,2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
2,3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
3,4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
4,5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [9]:
# 第一列是原来导出的index，可以删除
data.drop('index', axis=1, inplace=True)
data.head()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
1,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
2,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
3,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
4,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


### 重复值处理

检查重复值

In [10]:
data.shape

(150000, 11)

In [11]:
# 查看哪些行是重复的
# keep=False表示标识出所有的重复值，而不是只标识除第一个以外的值
dup_index = data.duplicated(keep=False)
# type(dup_index)
dup_index[dup_index]

52        True
331       True
530       True
1250      True
1669      True
          ... 
149139    True
149431    True
149472    True
149769    True
149993    True
Length: 960, dtype: bool

In [12]:
# 重复的行数，这里包括的重复值本身
dup_index.sum()

960

删除重复值

In [13]:
data.drop_duplicates(inplace=True)
data.shape

(149391, 11)

### 缺失值处理

检查缺失值

In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 149391 entries, 0 to 149999
Data columns (total 11 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   SeriousDlqin2yrs                      149391 non-null  int64  
 1   RevolvingUtilizationOfUnsecuredLines  149391 non-null  float64
 2   age                                   149391 non-null  int64  
 3   NumberOfTime30-59DaysPastDueNotWorse  149391 non-null  int64  
 4   DebtRatio                             149391 non-null  float64
 5   MonthlyIncome                         120170 non-null  float64
 6   NumberOfOpenCreditLinesAndLoans       149391 non-null  int64  
 7   NumberOfTimes90DaysLate               149391 non-null  int64  
 8   NumberRealEstateLoansOrLines          149391 non-null  int64  
 9   NumberOfTime60-89DaysPastDueNotWorse  149391 non-null  int64  
 10  NumberOfDependents                    145563 non-null  float64
dtype

In [15]:
# 检查每个特征的缺失值数量
data.isnull().sum()

SeriousDlqin2yrs                            0
RevolvingUtilizationOfUnsecuredLines        0
age                                         0
NumberOfTime30-59DaysPastDueNotWorse        0
DebtRatio                                   0
MonthlyIncome                           29221
NumberOfOpenCreditLinesAndLoans             0
NumberOfTimes90DaysLate                     0
NumberRealEstateLoansOrLines                0
NumberOfTime60-89DaysPastDueNotWorse        0
NumberOfDependents                       3828
dtype: int64

In [16]:
# 检查每个特征的缺失值占比
data.isnull().mean()*100

SeriousDlqin2yrs                         0.000000
RevolvingUtilizationOfUnsecuredLines     0.000000
age                                      0.000000
NumberOfTime30-59DaysPastDueNotWorse     0.000000
DebtRatio                                0.000000
MonthlyIncome                           19.560081
NumberOfOpenCreditLinesAndLoans          0.000000
NumberOfTimes90DaysLate                  0.000000
NumberRealEstateLoansOrLines             0.000000
NumberOfTime60-89DaysPastDueNotWorse     0.000000
NumberOfDependents                       2.562403
dtype: float64

从上面可以看出，MonthlyIncome和NumberOfDependents这两个特征有缺失值。  
月收入这个指标缺失值占比较高，但是不能随便删除，因为和业务的关系很大；  
家庭成员个数可以考虑填补。

**填补家庭成员特征，使用均值填补**

In [17]:
# 这里填补的时候要注意转成整型
data['NumberOfDependents'].fillna(int(data['NumberOfDependents'].mean()), inplace=True)

In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 149391 entries, 0 to 149999
Data columns (total 11 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   SeriousDlqin2yrs                      149391 non-null  int64  
 1   RevolvingUtilizationOfUnsecuredLines  149391 non-null  float64
 2   age                                   149391 non-null  int64  
 3   NumberOfTime30-59DaysPastDueNotWorse  149391 non-null  int64  
 4   DebtRatio                             149391 non-null  float64
 5   MonthlyIncome                         120170 non-null  float64
 6   NumberOfOpenCreditLinesAndLoans       149391 non-null  int64  
 7   NumberOfTimes90DaysLate               149391 non-null  int64  
 8   NumberRealEstateLoansOrLines          149391 non-null  int64  
 9   NumberOfTime60-89DaysPastDueNotWorse  149391 non-null  int64  
 10  NumberOfDependents                    149391 non-null  float64
dtype

**填补月收入，这里使用随机森林填补**

In [27]:
def fill_missing_rf(X, y, to_fill, n_estimators=100):
    """
    """
    df = X.copy()
    column_to_fill = df[to_fill]
    df = pd.concat([df.loc[:, df.columns!=to_fill], pd.DataFrame(y)], axis=1)
    # 找出训练集和测试集
    Xtrain = df.loc[column_to_fill.notnull(), :]
    Xfill = df.loc[column_to_fill.isnull(), :]
    Ytrain = column_to_fill[column_to_fill.notnull()]

    # 使用随机森林填补
    from sklearn.ensemble import RandomForestRegressor
    rfr = RandomForestRegressor(n_estimators=n_estimators)
    rfr.fit(Xtrain,Ytrain)
    Yfill = rfr.predict(Xfill)
    return Yfill

data中的第一列是表示目标变量，这里还没有分离，需要分开

In [20]:
X = data.iloc[:, 1:]
y = data.iloc[:, 0]

In [21]:
X.shape

(149391, 10)

In [28]:
fill = fill_missing_rf(X,y, to_fill='MonthlyIncome')

In [31]:
fill.shape

(29221,)

In [39]:
t = data.copy()
# 下面这种链式的取法会导致warning，原因是首先t['MonthlyIncome']取了DF里的一个切片，然后对这个切片进行了赋值
# 由于Pandas中的切片默认是视图，会导致原始DF中的列也被改变——这个操作有两种解释，一个是只想修改切片而不是DF里的值，另一个是修改DF里的值，
# 模棱两可
t['MonthlyIncome'][t['MonthlyIncome'].isnull()] = fill

/Users/danielzhang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


ValueError: Length of replacements must equal series length

In [38]:
t = data.copy()
# 但是使用下面这种方式就明确的指出了我们是要修改DF里了某一列，就不会报warning了
t.loc[t['MonthlyIncome'].isnull(), 'MonthlyIncome'] = fill

ValueError: Must have equal len keys and value when setting with an iterable

In [36]:
# 填补缺失值
data.loc[data['MonthlyIncome'].isnull(), 'MonthlyIncome'] = fill

In [40]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 149391 entries, 0 to 149999
Data columns (total 11 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   SeriousDlqin2yrs                      149391 non-null  int64  
 1   RevolvingUtilizationOfUnsecuredLines  149391 non-null  float64
 2   age                                   149391 non-null  int64  
 3   NumberOfTime30-59DaysPastDueNotWorse  149391 non-null  int64  
 4   DebtRatio                             149391 non-null  float64
 5   MonthlyIncome                         149391 non-null  float64
 6   NumberOfOpenCreditLinesAndLoans       149391 non-null  int64  
 7   NumberOfTimes90DaysLate               149391 non-null  int64  
 8   NumberRealEstateLoansOrLines          149391 non-null  int64  
 9   NumberOfTime60-89DaysPastDueNotWorse  149391 non-null  int64  
 10  NumberOfDependents                    149391 non-null  float64
dtype

### 异常值处理

In [41]:
data.describe([0.01,0.1,0.25,.5,.75,.9,.99]).T

,count,mean,std,min,1%,10%,25%,50%,75%,90%,99%,max
SeriousDlqin2yrs,149391.0,0.066999,0.250021,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.0
RevolvingUtilizationOfUnsecuredLines,149391.0,6.071087,250.263672,0.0,0.0,0.003199,0.030132,0.154235,0.556494,0.978007,1.093922,50708.0
age,149391.0,52.306237,14.725962,0.0,24.0,33.000000,41.000000,52.000000,63.000000,72.000000,87.000000,109.0
NumberOfTime30-59DaysPastDueNotWorse,149391.0,0.393886,3.852953,0.0,0.0,0.000000,0.000000,0.000000,0.000000,1.000000,4.000000,98.0
DebtRatio,149391.0,354.436740,2041.843455,0.0,0.0,0.034991,0.177441,0.368234,0.875279,1275.000000,4985.100000,329664.0
MonthlyIncome,149391.0,5424.877449,13238.469554,0.0,0.0,0.180000,1800.000000,4420.000000,7416.000000,10800.000000,23250.000000,3008750.0
NumberOfOpenCreditLinesAndLoans,149391.0,8.480892,5.136515,0.0,0.0,3.000000,5.000000,8.000000,11.000000,15.000000,24.000000,58.0
NumberOfTimes90DaysLate,149391.0,0.238120,3.826165,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,98.0
NumberRealEstateLoansOrLines,149391.0,1.022391,1.130196,0.0,0.0,0.000000,0.000000,1.000000,2.000000,2.000000,4.000000,54.0
NumberOfTime60-89DaysPastDueNotWorse,149391.0,0.212503,3.810523,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,98.0


age有异常

In [42]:
(data['age'] == 0).sum()

1

In [43]:
data.shape

(149391, 11)

In [44]:
data = data[data['age']!=0]

In [45]:
data.shape

(149390, 11)

NumberOfTimes90DaysLate有异常

In [46]:
data[data.loc[:,"NumberOfTimes90DaysLate"] > 90].count()

SeriousDlqin2yrs                        225
RevolvingUtilizationOfUnsecuredLines    225
age                                     225
NumberOfTime30-59DaysPastDueNotWorse    225
DebtRatio                               225
MonthlyIncome                           225
NumberOfOpenCreditLinesAndLoans         225
NumberOfTimes90DaysLate                 225
NumberRealEstateLoansOrLines            225
NumberOfTime60-89DaysPastDueNotWorse    225
NumberOfDependents                      225
dtype: int64

In [47]:
data = data[data.loc[:,"NumberOfTimes90DaysLate"] < 90]

In [48]:
data.shape

(149165, 11)

### 样本不平衡问题

In [50]:
X = data.iloc[:, 1:]
y = data.iloc[:,0]
y.value_counts()

0    139292
1      9873
Name: SeriousDlqin2yrs, dtype: int64

使用上采样的方法来平衡样本

In [51]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=29)
X, y = smote.fit_resample(X,y)

In [54]:
y.value_counts()

1    139292
0    139292
Name: SeriousDlqin2yrs, dtype: int64

In [55]:
type(X)

pandas.core.frame.DataFrame

In [56]:
type(y)

pandas.core.series.Series

### 划分训练集和测试集

In [58]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.3, random_state=29)

In [62]:
data_train = pd.concat([ytrain, Xtrain], axis=1)

In [63]:
data_train.head()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
222414,1,0.526111,57,2,1.048608,6000.000000,10,0,3,0,1.786411
79791,0,0.377635,56,1,0.683326,2200.000000,18,0,0,0,0.000000
68423,1,0.970913,57,0,0.698603,3506.000000,10,0,2,0,0.000000
225486,1,0.669247,47,0,0.418189,13960.930998,6,0,2,0,2.000000
219846,1,0.416925,48,1,0.435004,8365.393247,13,0,1,0,1.651688


In [64]:
data_val = pd.concat([ytest, Xtest], axis=1)

-----

## 分箱